In [ ]:
'''
A unique approach to data pre-processing of train description data was used, allowing the data to be adapted to the model.
'''

In [2]:
import pandas as pd 
import numpy as np

In [2]:
original_data = pd.read_csv(r"F:\document\Dissertation\open-data\open-data2.csv",index_col= 0)
original_data

,Timestamp,Type,ID,Event2,State,Panel_Request,From_Berth,To_Berth
Event,,,,,,,,
1,18/10/2019 10:42,Signal,STD5053,STD5053,0,NaN,NaN,NaN
2,18/10/2019 10:42,Track,TNGM,TNGM,1,NaN,NaN,NaN
3,18/10/2019 10:42,CA,B5313,CAB5053B5313,1K11,NaN,B5053,B5313
4,18/10/2019 10:42,CC,BABDT,CCBABDT,1K11,NaN,NaN,BABDT
5,18/10/2019 10:42,Track,TDCF,TDCF,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
991458,08/11/2019 13:31,Track,TDMF,TDMF,1,NaN,NaN,NaN
991459,08/11/2019 13:31,Track,TDMC,TDMC,0,NaN,NaN,NaN
991460,08/11/2019 13:31,Track,TNGF,TNGF,1,NaN,NaN,NaN


In [2]:
#Some of the train_ID data is wrong. The definition function modifies the incorrect trainID.
def change_train_id(data):
    data_state_list=data['State'].tolist()
    add_ts=[]
    for i in data_state_list:
        if len(i)!=4 and len(i)!=1  :
            add_ts.append(i[0]+i[4]+i[6]+i[7])
        else:
            add_ts.append(i)
    data['State']=add_ts
    return data

In [3]:
#Define functions to remove CB,CC,Track,TRTS data and convert the route data with status 1 to panel request.
def get_new_clear_data(data):
    data2=data[data['Type'].isin(['Signal','CA','Route'])]
    
    type_old=data2['Type'].tolist()
    state=data2['State'].tolist()
    new_type=[]
    for i in range(0,len(type_old)):
        if type_old[i]=='Route' and state[i]=='1':
            new_type.append('Panel Request')
        else:
            new_type.append(type_old[i])
            
    data2['Type']=new_type
    return data2

In [5]:
new_data=change_train_id(original_data)
new_clear_data=get_new_clear_data(new_data)
new_clear_data

<ipython-input-4-01004123f3dc>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Type']=new_type


,Timestamp,Type,ID,Event2,State,Panel_Request,From_Berth,To_Berth
Event,,,,,,,,
1,18/10/2019 10:42,Signal,STD5053,STD5053,0,NaN,NaN,NaN
3,18/10/2019 10:42,CA,B5313,CAB5053B5313,1K11,NaN,B5053,B5313
8,18/10/2019 10:42,CA,B0556,CAB0554B0556,1F20,NaN,B0554,B0556
14,18/10/2019 10:42,Route,RDC5065C(M),RDC5065C(M),0,NaN,NaN,NaN
16,18/10/2019 10:42,Signal,STD5032,STD5032,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
991449,08/11/2019 13:31,Panel Request,RDW5306A(C),RDW5306A(C),1,NaN,NaN,NaN
991453,08/11/2019 13:31,Signal,STD5029,STD5029,0,NaN,NaN,NaN
991455,08/11/2019 13:31,CA,B5033,CAB5029B5033,1V10,QRTD5045B-2(M),B5029,B5033


In [ ]:
#Define functions to get the state of the signal

In [5]:
# sorting signal
def sort_signal_ID(data):
    Signal=data[data['Type']=='Signal']
    Signal_ID=Signal['ID'].tolist()
    Signal_new_ID=[]
    for i in Signal_ID:
        if i[0:3]=='SDY':
            Signal_new_ID.append('0'+i[3:])
        else:
            Signal_new_ID.append(i[3:])
            
    Signal.drop(["From_Berth","To_Berth","Timestamp","Event2","Panel_Request","Type"],inplace=True,axis=1) 
    Signal['new_ID']=Signal_new_ID
    Signal['index']=Signal.index
    after_sort_signal=Signal.sort_values(by = ['new_ID','index'],axis = 0,ascending = True)
    after_sort_signal.drop(["new_ID","index"],inplace=True,axis=1)
    return after_sort_signal

In [6]:
#Define functions to get the initial state of all signals.
def get_initial_state(after_sort_signal):
    after_sort_signal.index = range(len(after_sort_signal))
    after_sort_signal['is_duplicated'] = after_sort_signal.duplicated(['ID'])
    data2=after_sort_signal[after_sort_signal['is_duplicated']==False]
    data2.index = range(len(data2))
    data2.drop(["is_duplicated"],inplace=True,axis=1) 
    data_initial=data2.copy()
    
    state1=data_initial['State'].tolist()
    state2=[]
    for i in state1:
        if i=='0':
            state2.append('1')
        else:
            state2.append('0')
            
    data_initial['initial_state']=state2
    data_initial.drop(["State"],inplace=True,axis=1) 
    data_initial=data_initial.T
    data_initial.columns=data_initial.iloc[0].tolist()
    data_initial.drop(index='ID',inplace=True)
    data_initial.index=[1]
    return data_initial

In [6]:
#Update the state of the signal as the data changes, wrapping it as a function to get all the states.

def get_signal_state(data,after_sort_signal):
    Signal=data[data['Type']=='Signal']
    Signal.drop(["From_Berth","To_Berth","Timestamp","Type","Event2","Panel_Request"],inplace=True,axis=1) 
    old_index=Signal.index.copy() 
    Signal.index = range(len(Signal))
    Signal['State'] = Signal['State'].astype(int)
    
    data_initial=get_initial_state(after_sort_signal)
    
    data_initial_sum=data_initial.copy()
    data_initial_change=data_initial.copy()
    
    data_initial_sum = []
    for i in Signal.index:
        data_initial_change[Signal['ID'][i]].replace(data_initial_change[Signal['ID'][i]][1],Signal['State'][i],inplace=True)
        data_initial_sum.extend(data_initial_change.values.tolist())
    pd_data_initial_sum = pd.DataFrame(data_initial_sum)
    pd_data_initial_sum.columns=data_initial.columns
    pd_data_initial_sum.index=old_index
    return pd_data_initial_sum

In [7]:
# Define functions to get all the state
def get_datawithstate(new_clear_data):
    data_withstate=new_clear_data.copy()
    data_withstate.index=range(1,len(data_withstate)+1)
    
    after_sort_signal=sort_signal_ID(data_withstate)
    
    
    pd_data_initial_sum=get_signal_state(data_withstate,after_sort_signal)
    
    ## Add status to all data
    signalstate=pd_data_initial_sum
    ssi=signalstate.index.tolist()
    length=signalstate.index[-1]+1

    fullindex=[]
    for i in range(1,length):
        if i not in ssi:
            fullindex.append(fullindex[i-2])
        else:
            fullindex.append(i)

    fullss=[]
    for i in fullindex:
        fullss.append(signalstate.loc[i].values.tolist())

    pdfullss=pd.DataFrame(fullss)
    pdfullss.columns=signalstate.columns
    pdfullss.index=new_clear_data.index
    datawithstate=pd.concat([new_clear_data,pdfullss],axis=1)
    return datawithstate

In [34]:
datawithstate=get_datawithstate(new_clear_data)
datawithstate

<ipython-input-30-8786c7fb672f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Signal['new_ID']=Signal_new_ID
<ipython-input-30-8786c7fb672f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Signal['index']=Signal.index
<ipython-input-32-b271d1ea8d0a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

,Timestamp,Type,ID,Event2,State,Panel_Request,From_Berth,To_Berth,SDY544,SDY551,...,SEC5488,SEC5489,SEC5490,SEC5491,SEC5493,SEC5496,SEC5501,SEC5502,SEC5503,SEC5504
Event,,,,,,,,,,,,,,,,,,,,,
1,18/10/2019 10:42,Signal,STD5053,STD5053,0,NaN,NaN,NaN,1,1,...,0,0,0,0,0,0,0,0,0,0
3,18/10/2019 10:42,CA,B5313,CAB5053B5313,1K11,NaN,B5053,B5313,1,1,...,0,0,0,0,0,0,0,0,0,0
8,18/10/2019 10:42,CA,B0556,CAB0554B0556,1F20,NaN,B0554,B0556,1,1,...,0,0,0,0,0,0,0,0,0,0
14,18/10/2019 10:42,Route,RDC5065C(M),RDC5065C(M),0,NaN,NaN,NaN,1,1,...,0,0,0,0,0,0,0,0,0,0
16,18/10/2019 10:42,Signal,STD5032,STD5032,1,NaN,NaN,NaN,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991449,08/11/2019 13:31,Panel Request,RDW5306A(C),RDW5306A(C),1,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
991453,08/11/2019 13:31,Signal,STD5029,STD5029,0,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
991455,08/11/2019 13:31,CA,B5033,CAB5029B5033,1V10,QRTD5045B-2(M),B5029,B5033,1,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#The state of CA data is the train_ID , which is added to the feature train_ID.
def get_ca_trainID_toadd(data):
    
    CA=data[data["Type"]=='CA']
    ca_state_list=CA['State'].tolist()
    
    caadd=[]
    each=[]

    for i in range(0,len(ca_state_list)):
        each=[CA.index[i],ca_state_list[i]]
        caadd.append(each) 
        
    return caadd      

In [12]:
## Add the acquired train ID to the featured train_ID
def add_to_data(caadd,data2,data):
    pa_CA_ADD = pd.DataFrame(caadd)
    pa_CA_ADD.set_index([0], inplace=True)
    
    caold=data2['Train_ID']
    caold=pd.DataFrame(caold)
    caold.dropna(subset=['Train_ID'],inplace=True)
    catoadd=pa_CA_ADD.join(caold,how='outer')
    
    list1=catoadd[1].tolist()
    list2=catoadd['Train_ID'].tolist()
    list3=[]
    for i in range(0,len(list1)):
        if list1[i] is np.nan:
            list3.append(list2[i])
        if list2[i] is np.nan:
            list3.append(list1[i])
        if list1[i] is not np.nan and list2[i] is not np.nan:
            list3.append(list1[i])   # If there are special circumstances that choose to believe here
    
    catoadd['new']=list3
    catoadd.drop([1,'Train_ID'],inplace=True,axis=1)
    catoadd.rename(columns={'new':'Train_ID'}, inplace = True)
    
    data.drop(['Train_ID'],inplace=True,axis=1)
    datawithca = pd.merge(data,catoadd,how='outer',left_index=True,right_index=True)
    
    return datawithca 

In [ ]:
#Find the train ID for the PR.

In [8]:
def get_caadd_0(new_clear_data):
    pr= new_clear_data[new_clear_data['Type']=='Panel Request']
    pridlist=[]
    for i in pr['ID'].tolist():
        if i[0:3]=='RDY':
            pridlist.append('0'+i[3:6])
        else:
            pridlist.append(i[3:7])
            
    CA=new_clear_data[new_clear_data['Type']=='CA']


    def moveb(a):
        b=[]
        for i in a:
            b.append(i[1:5])
        return b

    pr_train_id=[]
    each=[]
    for i in range(0,len(pridlist)):
        j = pr.index[i]
        fbl= moveb(CA.loc[j:]['From_Berth'].tolist())
        if pridlist[i] in fbl:
            place=CA.loc[j:].iloc[fbl.index(pridlist[i])]
            each=[place.name,j,place.State]
            pr_train_id.append(each)


    #It's not sure if it's just the nearest or both. Here, just the nearest for now.
    reverse_prtid=[]
    for i in reversed(pr_train_id):
        reverse_prtid.append(i)



    reverse_fin_prtid=[]

    reverse_fin_prtid.append(reverse_prtid[0])
    for j in range(1,len(reverse_prtid)):
        eachchange=[]
        for k in reverse_fin_prtid:
            eachchange.append(k[0])
        if reverse_prtid[j][0] not in eachchange:
            reverse_fin_prtid.append(reverse_prtid[j])


    fin_prtid=[]
    for m in reversed(reverse_fin_prtid):
        fin_prtid.append(m)

    pa_CA_ADD = pd.DataFrame(fin_prtid)
    pa_CA_ADD.set_index([1], inplace=True)
    pa_CA_ADD.drop([0], inplace=True,axis=1)
    
    pa_CA_ADD.rename(columns={2:'Train_ID'}, inplace = True)
    
    datawithca0 = pd.merge(new_clear_data,pa_CA_ADD,how='outer',left_index=True,right_index=True)
    
    return datawithca0

In [ ]:
# Add data to feature train_ID

In [9]:
# to list
def get_datalist(data):
    data2=data[data['Type'].isin(['Signal', 'Route','CA'])]
    
    dtype=data2['Type'].tolist()
    ds=data2['State'].tolist()
    dctb=data2['To_Berth'].tolist()
    did=data2['ID'].tolist()
    did_new=[]
    for i in range(0,len(did)):
        if dtype[i]=='Route' and did[i][0:3]=='RDY':
            did_new.append('0'+did[i][3:6])
        elif dtype[i]=='Signal' and did[i][0:3]=='SDY':
            did_new.append('0'+did[i][3:6])
        elif dtype[i]=='CA':
            did_new.append(did[i][1:5])
        else:
            did_new.append(did[i][3:7])
    
    datalist=[]
    each=[]
    for i in range(0,len(did_new)):
        if dtype[i]=='CA':
            each=[data2.index[i],dtype[i],dctb[i][1:5],ds[i]]
            datalist.append(each)
        else:    
            each=[data2.index[i],dtype[i],did_new[i],ds[i]]
            datalist.append(each)
            
    return datalist

In [10]:
def get_caadd_1(data):
    datalist=get_datalist(data)
   # calist=get_calist(data)
    
    caadd=[]
    each=[]
    kk=[]
    gg=[]
    mm=[]
    dd=[]
    ss=[]

    for i in range(0,len(datalist)):
        mm.clear()
        kk.clear()
        dd.clear()
        ss.clear()
        gg.clear()

        if datalist[i][1]=='CA':
            kk=datalist[i+1:]
            for l in range(0,len(kk)):
                if kk[l][2]==datalist[i][2]:  # Take data with the same ID
                    dd.append(kk[l])

            for j in range(0,len(dd)):
                if dd[j][1]=='CA':
                    break
                mm.append(dd[j])

            for k in range(0,len(mm)-1):
                if mm[k][1]=='Signal' and mm[k][3]=='0' and ((mm[k+1][1]=='Signal' and mm[k+1][3]=='1') or mm[k+1][1]=='Route'):
                    ss.append(mm[k])
                    gg.append(mm[k+1])
                #elif  mm[k][1]=='Route' and mm[k][3]=='0' and  mm[k+1][1]=='Signal' and mm[k+1][3]=='0':
                #    ss.append(mm[k+1])
                #    gg.append(mm[k])
                else:
                    continue

            if (len(ss)>0) and (len(gg)>0):
                each=[ss[0][0],datalist[i][3]]     
                caadd.append(each)
                each=[gg[0][0],datalist[i][3]]     
                caadd.append(each)

    return caadd    

In [ ]:
# Add trainID to the signal below pr

In [13]:
#Gets a signalID that issues pr
def get_pr_signalID(data):
    pr=data[data['Type']=='Panel Request']
    pridlist=[]
    
    for i in pr['ID'].tolist():
        if i[0:3]=='RDY':
            pridlist.append('0'+i[3:6])
        else:
            pridlist.append(i[3:7])
            
    pr_signalID=(list(set(pridlist)))
    
    return pr_signalID

In [14]:
def get_datalist_with_tid(data):
    data2=data.copy()
    
    dtype=data2['Type'].tolist()
    ds=data2['State'].tolist()
    dctb=data2['To_Berth'].tolist()
    did=data2['ID'].tolist()
    dtid=data2['Train_ID'].tolist()
    
    did_new=[]
    for i in range(0,len(did)):
        if dtype[i]=='Route' and did[i][0:3]=='RDY':
            did_new.append('0'+did[i][3:6])
        elif dtype[i]=='Signal' and did[i][0:3]=='SDY':
            did_new.append('0'+did[i][3:6])
        elif dtype[i]=='Panel Request' and did[i][0:3]=='RDY':
            did_new.append('0'+did[i][3:6])
        elif dtype[i]=='CA':
            did_new.append(did[i][1:5])
        else:
            did_new.append(did[i][3:7])
    
    datalist=[]
    each=[]
    for i in range(0,len(did_new)):
        if dtype[i]=='CA':
            each=[data2.index[i],dtype[i],dctb[i][1:5],ds[i],dtid[i]]
            datalist.append(each)
        else:    
            each=[data2.index[i],dtype[i],did_new[i],ds[i],dtid[i]]
            datalist.append(each)
            
    return datalist

In [15]:
def get_caadd_2(data):

    datalist_with_tid=get_datalist_with_tid(data)   
    pr_signalID=get_pr_signalID(data)   

    caadd=[]
    for i in pr_signalID:
        sub_data=[]
        for j in datalist_with_tid:
            if j[2]==i:
                sub_data.append(j)

        for k in range(0,len(sub_data)-2):
            if sub_data[k][1]=='Panel Request' and (sub_data[k+1][1]=='Signal' and sub_data[k+1][3]=='1') and (sub_data[k][4] is not np.nan):
                each=[sub_data[k+1][0],sub_data[k][4]]
                caadd.append(each)
            elif sub_data[k][1]=='Panel Request' and (sub_data[k-1][1]=='Signal' and sub_data[k-1][3]=='1') and (sub_data[k][4] is not np.nan):
                each=[sub_data[k-1][0],sub_data[k][4]]
                caadd.append(each)
            elif sub_data[k][1]=='Panel Request' and (sub_data[k+2][1]=='Signal' and sub_data[k+2][3]=='1') and sub_data[k+1][1]=='CA' and (sub_data[k][4] is not np.nan):
                each=[sub_data[k+2][0],sub_data[k][4]]
                caadd.append(each)
                
    return caadd
        

In [16]:
# Get the train ID for all the data
## Must be followed in this order, as what follows complements and corrects what comes before.

def get_datawithca(new_clear_data):
    
    datawithca0=get_caadd_0(new_clear_data)
    
    basic_data=datawithca0.copy()  
    caadd_1=get_caadd_1(basic_data)
    datawithca1=add_to_data(caadd_1,basic_data,basic_data)
    
    basic_data_2=datawithca0.copy()
    ca_trainID_toadd=get_ca_trainID_toadd(basic_data_2)
    datawithca1_1=add_to_data(ca_trainID_toadd,datawithca1,basic_data_2)

    basic_data_3=datawithca0.copy()
    caadd_2=get_caadd_2(basic_data_3)
    datawithca_2=add_to_data(caadd_2,datawithca1_1,basic_data_3)
    
    return datawithca_2
    

In [19]:
datawithca=get_datawithca(new_clear_data)
datawithca

,Timestamp,Type,ID,Event2,State,Panel_Request,From_Berth,To_Berth,Train_ID
1,18/10/2019 10:42,Signal,STD5053,STD5053,0,NaN,NaN,NaN,NaN
3,18/10/2019 10:42,CA,B5313,CAB5053B5313,1K11,NaN,B5053,B5313,1K11
8,18/10/2019 10:42,CA,B0556,CAB0554B0556,1F20,NaN,B0554,B0556,1F20
14,18/10/2019 10:42,Route,RDC5065C(M),RDC5065C(M),0,NaN,NaN,NaN,NaN
16,18/10/2019 10:42,Signal,STD5032,STD5032,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
991449,08/11/2019 13:31,Panel Request,RDW5306A(C),RDW5306A(C),1,NaN,NaN,NaN,NaN
991453,08/11/2019 13:31,Signal,STD5029,STD5029,0,NaN,NaN,NaN,NaN
991455,08/11/2019 13:31,CA,B5033,CAB5029B5033,1V10,QRTD5045B-2(M),B5029,B5033,1V10
991456,08/11/2019 13:31,Panel Request,RTD5045B-2(M),RTD5045B-2(M),1,NaN,NaN,NaN,NaN


In [17]:
def get_full_data(original_data):
    # Obtaining basic data
    new_data=change_train_id(original_data)
    new_clear_data=get_new_clear_data(new_data)
    
    # Get the status of full data
    datawithstate=get_datawithstate(new_clear_data)
    
    # Get feature train_ID
    datawithca=get_datawithca(new_clear_data)
    
    # Add status and train ID to the data
    state=datawithstate.iloc[:,8:]
    state.index=datawithca.index 
    full_data=pd.concat([datawithca,state],axis=1)
    return full_data

In [ ]:
#Save the processed data
full_data.to_csv(r"full_data_with_state.csv")